In [1]:
from bancor_research import DEFAULT
from bancor_research.bancor_simulator.v3.spec.network import BancorDapp as sBancorDapp
from bancor_research.bancor_emulator.v3.spec.network import BancorDapp as eBancorDapp

# Recall from an earlier chapter that we already defined the whitelisted_tokens as follows.
whitelisted_tokens = {tkn_name : DEFAULT.WHITELIST[tkn_name] for tkn_name in DEFAULT.WHITELIST if tkn_name in ['eth', 'link', 'wbtc']}

# There are other possible configuration settings available, however for the present purpose we will use the defaults.
dapps = [BancorDapp(whitelisted_tokens=whitelisted_tokens) for BancorDapp in [sBancorDapp, eBancorDapp]]

def assertAndDisplay(decimals = -1):
    frames = [v3.describe(decimals) for v3 in dapps]
    diff = frames[0].compare(frames[1])
    assert diff.empty, diff
    return frames[0]

for v3 in dapps:
    v3.create_user('Trader')

    v3.set_user_balance(tkn_amt='101', tkn_name='eth', user_name='Alice')
    v3.set_user_balance(tkn_amt='101', tkn_name='wbtc', user_name='Bob')
    v3.set_user_balance(tkn_amt='10001', tkn_name='link', user_name='Charlie')
    v3.set_user_balance(tkn_amt='2000', tkn_name='bnt', user_name='Trader')

    v3.deposit(tkn_amt='100', tkn_name='eth', user_name='Alice')
    v3.deposit(tkn_amt='10000', tkn_name='link', user_name='Charlie')
    v3.deposit(tkn_amt='100', tkn_name='wbtc', user_name='Bob')
    v3.deposit(tkn_amt='1', tkn_name='eth', user_name='Alice')
    v3.deposit(tkn_amt='1', tkn_name='link', user_name='Charlie')
    v3.deposit(tkn_amt='1', tkn_name='wbtc', user_name='Bob')

    for tkn_name in whitelisted_tokens:
        v3.enable_trading(tkn_name)

# Trading and Fees - BNT to TKN

Continuing from the system state described above, the details of how the trading pools are used as a price oracle, the
behavior and purpose of the price EMA, and the response of the staking ledger to the accumulation of swap revenue will
now be discussed in detail. For the purpose of demonstration, let each of these pools have a 1% swap fee, and a Vortex
rate of 20%.

In [2]:
assertAndDisplay(decimals=6)

bnt           eth          link  \
1 Account Alice                         0.000000      0.000000      0.000000   
          Bob                           0.000000      0.000000      0.000000   
          Charlie                       0.000000      0.000000      0.000000   
          Trader                     2000.000000      0.000000      0.000000   
2 Pool    a: TKN Staked Balance                0    101.000000  10001.000000   
          b: TKN Trading Liquidity             0     20.000000   3333.333333   
          c: BNT Trading Liquidity             0  20000.000000  20000.000000   
          d: BNT Current Funding               0  20000.000000  20000.000000   
          e: Spot Rate                         0   1000.000000      6.000000   
          f: Average Rate                      0   1000.000000      6.000000   
          g: Average Inverse Rate              0      0.001000      0.166667   
3 Network BNT Staked Balance        60000.000000             0             0   
          Master Vault              60000.000000    101.000000  10001.000000   
          Protection Vault              0.000000      0.000000      0.000000   
          Protocol Equity                      0             0             0   
          Rewards Vault                        0             0             0   

                                            wbtc         bnbnt       bneth  \
1 Account Alice                         0.000000      0.000000  101.000000   
          Bob                           0.000000      0.000000    0.000000   
          Charlie                       0.000000      0.000000    0.000000   
          Trader                        0.000000      0.000000    0.000000   
2 Pool    a: TKN Staked Balance       101.000000             0           0   
          b: TKN Trading Liquidity      1.250000             0           0   
          c: BNT Trading Liquidity  20000.000000             0           0   
          d: BNT Current Funding    20000.000000             0           0   
          e: Spot Rate              16000.000000             0           0   
          f: Average Rate           16000.000000             0           0   
          g: Average Inverse Rate       0.000062             0           0   
3 Network BNT Staked Balance                   0             0           0   
          Master Vault                101.000000             0           0   
          Protection Vault              0.000000             0           0   
          Protocol Equity                      0  60000.000000    0.000000   
          Rewards Vault                        0      0.000000    0.000000   

                                          bnlink      bnwbtc      vbnt  
1 Account Alice                         0.000000    0.000000  0.000000  
          Bob                           0.000000  101.000000  0.000000  
          Charlie                   10001.000000    0.000000  0.000000  
          Trader                        0.000000    0.000000  0.000000  
2 Pool    a: TKN Staked Balance                0           0         0  
          b: TKN Trading Liquidity             0           0         0  
          c: BNT Trading Liquidity             0           0         0  
          d: BNT Current Funding               0           0         0  
          e: Spot Rate                         0           0         0  
          f: Average Rate                      0           0         0  
          g: Average Inverse Rate              0           0         0  
3 Network BNT Staked Balance                   0           0         0  
          Master Vault                         0           0         0  
          Protection Vault                     0           0         0  
          Protocol Equity               0.000000    0.000000         0  
          Rewards Vault                 0.000000    0.000000         0

Assume a trader swaps 2,000 BNT for LINK; from the perspective of the trader, 2,000 BNT are sent into the vault, and 300
LINK are removed from the vault and sent back to him. The change to the system state is much more involved. The change
in the vault balances agrees with the trader; 2,000 BNT is received, and 300 LINK was emitted. However, the change in
the available trading liquidity presents the first evidence of a consequence of the new design.

In [3]:
for v3 in dapps:
    v3.trade(tkn_amt='2000', source_token='bnt',  target_token='link', user_name='Trader', timestamp=0)
assertAndDisplay(decimals=4)

bnt         eth        link  \
1 Account Alice                         0.0000      0.0000      0.0000   
          Bob                           0.0000      0.0000      0.0000   
          Charlie                       0.0000      0.0000      0.0000   
          Trader                        0.0000      0.0000    300.0000   
2 Pool    a: TKN Staked Balance              0    101.0000  10003.4242   
          b: TKN Trading Liquidity           0     20.0000   3033.3333   
          c: BNT Trading Liquidity           0  20000.0000  21995.6044   
          d: BNT Current Funding             0  20000.0000  20000.0000   
          e: Spot Rate                       0   1000.0000      7.2513   
          f: Average Rate                    0   1000.0000      6.0000   
          g: Average Inverse Rate            0      0.0010      0.1667   
3 Network BNT Staked Balance        60000.0000           0           0   
          Master Vault              62000.0000    101.0000   9701.0000   
          Protection Vault              0.0000      0.0000      0.0000   
          Protocol Equity                    0           0           0   
          Rewards Vault                      0           0           0   

                                          wbtc       bnbnt     bneth  \
1 Account Alice                         0.0000      0.0000  101.0000   
          Bob                           0.0000      0.0000    0.0000   
          Charlie                       0.0000      0.0000    0.0000   
          Trader                        0.0000      0.0000    0.0000   
2 Pool    a: TKN Staked Balance       101.0000           0         0   
          b: TKN Trading Liquidity      1.2500           0         0   
          c: BNT Trading Liquidity  20000.0000           0         0   
          d: BNT Current Funding    20000.0000           0         0   
          e: Spot Rate              16000.0000           0         0   
          f: Average Rate           16000.0000           0         0   
          g: Average Inverse Rate       0.0001           0         0   
3 Network BNT Staked Balance                 0           0         0   
          Master Vault                101.0000           0         0   
          Protection Vault              0.0000           0         0   
          Protocol Equity                    0  60000.0000    0.0000   
          Rewards Vault                      0      0.0000    0.0000   

                                        bnlink    bnwbtc    vbnt  
1 Account Alice                         0.0000    0.0000  0.0000  
          Bob                           0.0000  101.0000  0.0000  
          Charlie                   10001.0000    0.0000  0.0000  
          Trader                        0.0000    0.0000  0.0000  
2 Pool    a: TKN Staked Balance              0         0       0  
          b: TKN Trading Liquidity           0         0       0  
          c: BNT Trading Liquidity           0         0       0  
          d: BNT Current Funding             0         0       0  
          e: Spot Rate                       0         0       0  
          f: Average Rate                    0         0       0  
          g: Average Inverse Rate            0         0       0  
3 Network BNT Staked Balance                 0         0       0  
          Master Vault                       0         0       0  
          Protection Vault                   0         0       0  
          Protocol Equity               0.0000    0.0000       0  
          Rewards Vault                 0.0000    0.0000       0

While the vault balance of BNT was increased by the expected amount, the change in the available trading liquidity is less than this amount. The difference is due to the effect of the Bancor Vortex, which confiscates a portion of trade revenue as a form of insurance premium, that helps to stabilise the BNT economy. In this case, the fee apparent to the trader is 0.303 LINK; the Bancor protocol awards 80% of this amount to LINK liquidity providers (i.e. 0.2424 LINK) by simply iterating the staking ledger. The remaining 0.0606 LINK are used to perform a reverse swap, and purchase BNT atomically at the tail end of the trade. This BNT is removed from the trading liquidity, but it continues to reside in the vault. The fraction of the BNT tokens belonging to the vortex is recorded on the Vortex Ledger; the fate of these BNT tokens is discussed in detail in a later section.

Therefore, during the trade the following components of the system were changed:

1. The vault balances. The change in the vault is always in agreement with the trader's intuition. In this case, the trader sent 2,000 BNT into the vault, and received 300 LINK from it. Therefore, the vault balance of BNT must have increased by 2,000 BNT, and decreased by 300 LINK.

2. The staking ledger. Trading fees are always taken from the target asset. Therefore, the value of the bnLINK pool token must be appreciated by 80% of the fee apparent to the trader (i.e. the total fee - vortex rate). A total of 2.42424 LINK tokens are added to the staked amount, which becomes the property of the bnLINK pool token holders.

3. The vortex ledger. One fifth of the fee paid by the trader is collected, and swapped back to BNT. The BNT purchased during this step is left where it is inside the vault, and the amount is added to the vortex ledger.

The trading liquidity. After accounting for the effect of the collection by the vortex, and the swap back to BNT, the reason for the changes in the trading liquidity to appear as they do, become apparent.

The change in the BNT and LINK trading liquidity can be expressed as follows:

<p align="center">
  <img style="filter: invert(1) hue-rotate(180deg);" src="https://lh3.googleusercontent.com/kk9N2rYg_vT85LUHgA_RNrg04kHac4We8KvNBtV3_dw0K1q6LcAfJ7_NF3R5h1JEsuT5pn5S0Azi5vQ5PaFoHxMw3StkoADFpyKN6EQiXt1vON1gbbhfi5ayDraD2U8vE5FNbgCw" alt="">
</p>

<p align="center">
  <img style="filter: invert(1) hue-rotate(180deg);" src="https://lh3.googleusercontent.com/sg_YY1cyR6IsWBKfiUbj5v9rREz9A7PO8FmpK8uPov7SmyoYrK4H6FroNDZ7EOJ9LcC0IOVggaLW4J3rVx1rwrEzl0Aqd55Ec_5d_cXQwNhjWlWNO73Dr3XJq_wBAoXvdC_i3gTf" alt="">
</p>

<p align="center">
  <img style="filter: invert(1) hue-rotate(180deg);" src="https://lh4.googleusercontent.com/SvZOi1YaY_Bw_oNVmqje8I5GK2qfhh_e-ZYJpFrFAuMv94vB56IfTqE_cQT5ZXH5V-zAMV0ZSoVGFBK6UQL13xmF_RGha6aWDGHtxtyain_XDaYcmqHOdOrp-JW04h2pJ6-IUzk0" alt="">
</p>

where a1† and a1 are the BNT trading liquidity balances of the pool after and before the trade, respectively, b1† and b1 are the LINK trading liquidity balances of the pool after and before the trade, respectively, d1 is the pool fee (e.g. 0.01, or 1%) and e is the Vortex rate (e.g. 0.2, or 20%). The number of BNT tokens being sent into the vault is x, and the number of LINK tokens sent back to the trader is tknOut. The change in the TKN trading liquidity (and therefore the amount of TKN received by the trader) is unchanged from the standard case. The calculation of the fee awarded to liquidity providers, denominated in its own TKN and added to the staking ledger is:

<p align="center">
  <img style="filter: invert(1) hue-rotate(180deg);" src="https://lh5.googleusercontent.com/NUQMFTb0HxIOqjiTTDtdH6idb2J9MZ5yRQgWM-M-m64TXOkOfEnRRAHOeHVTGrqjC4N0eumSVWGuYOz_uDhJ81QpyHN_-u-IzQwxPf7PKdfp_oG6fOzz0_X7tejwut7RSn5Jb9oV" alt="">
</p>

And the calculation for the fee given to the Bancor Vortex, denominated exclusively in BNT and added to the vortex ledger is:

<p align="center">
  <img style="filter: invert(1) hue-rotate(180deg);" src="https://lh3.googleusercontent.com/ouesDSiw2euSrnst3f32ojpU4-XeZeZ6DcYdSOClV9YUiqbp9MAH8lWnz75j2v53GqjViQexkkDiOfQ_gWBEtkiyMztrFCvFVMdxqYRhnK0Vynew01aos3O3_GXzR8IYYjRLZKL6" alt="">
</p>